In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime

import sys
sys.path.insert(0, '../src/misc')
from sql_tools import *
from webscraping_tools import *
from tools import *


In [144]:
seasons = season_range(2017,2022)
master_boxscore = pd.DataFrame(read_database('database.db', f"'boxscore {seasons[0]}'"))
    
for season in seasons[1:]:
    master_boxscore = pd.concat([master_boxscore,read_database('database.db', f"'boxscore {season}'")], axis=0)
    
master_boxscore = master_boxscore.sort_values(by = ['game_date'],ascending=False).reset_index(drop=True)
    
master_odds = read_database('database.db', "odds").sort_values(by = ['game_date'],ascending=False).reset_index(drop=True)

In [140]:
master_boxscore_data_only = master_boxscore.copy()

column_ind = [i for i in range(0,len(master_boxscore_data_only.dtypes)) if master_boxscore_data_only.dtypes[i] == 'object']
master_boxscore_data_only = master_boxscore_data_only.drop(columns = master_boxscore_data_only.columns[column_ind], axis = 1)

master_boxscore_data_only 

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,48.0,58,96,0.604,27,49,0.551,14,16,0.875,...,1548,1188,1659,2440,2436,1643,1833,2114,1835,2210
1,48.0,42,99,0.424,14,37,0.378,16,22,0.727,...,837,1261,1908,1205,2052,700,2151,2260,803,2002
2,48.0,40,90,0.444,3,21,0.143,25,29,0.862,...,709,297,2107,2167,2132,1141,306,219,25,877
3,48.0,41,97,0.423,14,45,0.311,13,21,0.619,...,1548,1646,1529,1197,1176,1307,1547,1585,245,873
4,48.0,44,97,0.454,11,39,0.282,15,19,0.789,...,2203,2350,850,600,606,1350,2230,2044,459,1369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14113,48.0,38,76,0.500,7,21,0.333,25,30,0.833,...,654,1101,187,1849,742,656,633,558,3,604
14114,48.0,43,80,0.538,16,30,0.533,19,21,0.905,...,553,677,381,142,383,861,648,879,581,1358
14115,48.0,47,97,0.485,15,41,0.366,13,19,0.684,...,2118,1859,1681,804,830,1919,1720,1441,1063,1307
14116,48.0,36,88,0.409,8,32,0.250,19,25,0.760,...,689,1001,1649,762,1625,682,2101,2037,1063,1154


In [145]:
game_data_array = master_odds.values.tolist()

row_list=[]

for row in tqdm.tqdm(game_data_array):  
    team_temp = master_boxscore_data_only.loc[(master_boxscore['game_date'] <= row[1]) & (master_boxscore['teamAbbr']==row[2])][1:6].mean()
    team_temp.index ="team"+team_temp.keys()

    oppt_temp = master_boxscore_data_only.loc[(master_boxscore['game_date'] <= row[1]) & (master_boxscore['teamAbbr']==row[3])][1:6].mean()
    oppt_temp.index ="oppt"+oppt_temp.keys()

    row_list.append(pd.concat([team_temp, oppt_temp]).to_dict())

100%|██████████| 7528/7528 [00:22<00:00, 332.47it/s]


In [149]:
trailing_dataset = master_odds.join(pd.DataFrame(row_list)).dropna()